In [ ]:
%%capture
!pip install satpy pyorbital pyspectral

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('colab')
  !wget -q https://raw.githubusercontent.com/jhbravo/read_aws-cloud_abi_data/main/config_loader.py -O /usr/local/lib/python3.6/dist-packages/satpy/composites/config_loader.py
  !wget -q https://raw.githubusercontent.com/jhbravo/read_aws-cloud_abi_data/main/utils.py -O /usr/local/lib/python3.6/dist-packages/satpy/readers/utils.py
else:
  print('kaggle')
  !wget -q https://raw.githubusercontent.com/jhbravo/read_aws-cloud_abi_data/main/config_loader.py -O /opt/conda/lib/python3.7/site-packages/satpy/composites/config_loader.py
  !wget -q https://raw.githubusercontent.com/jhbravo/read_aws-cloud_abi_data/main/abi_base.py -O /opt/conda/lib/python3.7/site-packages/satpy/readers/utils.py

In [ ]:
from satpy import Scene
#from satpy.utils import debug_on
#debug_on()

In [ ]:
urls = ['https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B01_JP01_R10_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B02_JP01_R10_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B03_JP01_R05_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B04_JP01_R10_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B05_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B06_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B07_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B08_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B09_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B10_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B11_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B12_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B13_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B14_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B15_JP01_R20_S0101.DAT.bz2',
        'https://noaa-himawari8.s3.amazonaws.com/AHI-L1b-Japan/2021/02/11/0140/HS_H08_20210211_0140_B16_JP01_R20_S0101.DAT.bz2',]

In [ ]:
scn = Scene(reader='ahi_hsd', filenames=urls)

In [ ]:
scn.load([f'B{n:02d}' for n in range(1,17)])

In [ ]:
scn.show('B13')

In [ ]:
scn.all_composite_names()

In [ ]:
a2km = scn['B13'].area

In [ ]:
nscn = scn.resample(a2km, resampler='native')

In [ ]:
rgbim = 'airmass'
nscn.load([rgbim,])
nscn.show(rgbim)

In [ ]:
rgbim = 'water_vapors1'
nscn.load([rgbim,])
nscn.show(rgbim)

In [ ]:
rgbim = 'cloud_phase_distinction'
nscn.load([rgbim,])
nscn.show(rgbim)

In [ ]:
rgbim = 'true_color'
nscn.load([rgbim,])
nscn.show(rgbim)

In [ ]:
%%capture
## install cartopy source: https://github.com/googlecolab/colabtools/issues/85#issuecomment-709241391
# not using `python-cartopy` package as it doesn't include all build dependencies
!apt-get install libproj-dev proj-bin proj-data
!apt-get install libgeos-dev

# shapely needs to be reinstalled to use the same geos install as cartopy (https://github.com/SciTools/cartopy/issues/871)
!pip uninstall -y shapely
!pip install --no-binary shapely shapely
!pip install cartopy

In [ ]:
from satpy.writers import get_enhanced_image

import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=200)

cmap = ['#ffffff',] * 19 + ['#7f007f', '#8c0d87', '#99198e', '#a52696', '#b2339d', '#bf40a5', '#cc4cad', '#d959b4', '#e566bc', '#f272c3', '#ff7fcb', '#e6e6e6', '#cdcdcd',
        '#b3b3b3', '#9a9a9a', '#808080', '#676767', '#4d4d4d', '#343434', '#1a1a1a', '#000000', '#1a0000', '#330000', '#4d0000', '#660000', '#800000', '#990000', '#b30000', '#cc0000',
        '#e60000', '#ff0000', '#ff1a00', '#ff3300', '#ff4d00', '#ff6600', '#ff8000', '#ff9900', '#ffb300', '#ffcc00', '#ffe600', '#ffff00', '#e6ff00', '#ccff00', '#b3ff00', '#99ff00',
        '#80ff00', '#66ff00', '#4dff00', '#33ff00', '#1aff00', '#00ff00', '#00ea0a', '#00d513', '#00bf1d', '#00aa26', '#009530', '#00803a', '#006a43', '#00554d', '#004056', '#002b60',
        '#001569', '#000073', '#000073', '#000d7a', '#001a81', '#002688', '#00338f', '#004096', '#004d9d', '#0059a4', '#0066ab', '#0073b2', '#0080b9', '#008cc0', '#0099c7', '#00a6ce',
        '#00b3d5', '#00bfdc', '#00cce3', '#00d9ea', '#00e6f1', '#00f2f8', '#00ffff', '#ffffff', '#ffffff', '#ffffff', '#ffffff', '#ffffff', '#ffffff', '#ffffff', '#fefefe', '#fcfcfc',
        '#f9f9f9', '#f7f7f7', '#f4f4f4', '#f2f2f2', '#efefef', '#ededed', '#eaeaea', '#e8e8e8', '#e5e5e5', '#e2e2e2', '#e0e0e0', '#dddddd', '#dbdbdb', '#d8d8d8', '#d6d6d6', '#d3d3d3',
        '#d1d1d1', '#cecece', '#cbcbcb', '#c9c9c9', '#c6c6c6', '#c4c4c4', '#c1c1c1', '#bfbfbf', '#bcbcbc', '#bababa', '#b7b7b7', '#b5b5b5', '#b2b2b2', '#afafaf', '#adadad', '#aaaaaa',
        '#a8a8a8', '#a5a5a5', '#a3a3a3', '#a0a0a0', '#9e9e9e', '#9b9b9b', '#989898', '#969696', '#939393', '#919191', '#8e8e8e', '#8c8c8c', '#898989', '#878787', '#848484', '#828282',
        '#7f7f7f', '#7c7c7c', '#7a7a7a', '#777777', '#757575', '#727272', '#707070', '#6d6d6d', '#6b6b6b', '#686868', '#656565', '#636363', '#606060', '#5e5e5e', '#5b5b5b', '#595959',
        '#565656', '#545454', '#515151', '#4f4f4f', '#4c4c4c', '#494949', '#474747', '#444444', '#424242', '#3f3f3f', '#3d3d3d', '#3a3a3a', '#383838', '#353535', '#323232', '#303030',
        '#2d2d2d', '#2b2b2b', '#282828', '#262626', '#232323', '#212121', '#1e1e1e', '#1c1c1c', '#191919', '#161616', '#141414', '#111111', '#0f0f0f', '#0c0c0c', '#0a0a0a', '#070707',
        '#050505', '#020202', ] + ['#000000',] * 46

levels = np.linspace(-109+237.15, 56+237.15, num=len(cmap))
norm = plt.cm.colors.BoundaryNorm(levels, len(levels))
irmap = plt.cm.colors.ListedColormap(cmap)

var = nscn['B13']
strs = var.attrs
abi_crs = var.area.to_cartopy_crs()
ax = fig.add_subplot(1, 1, 1, projection=abi_crs)
ir_img = ax.imshow(var, origin='upper', extent=(var.x[0], var.x[-1], var.y[-1], var.y[0]), cmap=irmap)

#ax.set_extent([-123.0, -74.5, 10.0, 37.0]) #cambiar a area geografica de interes

ax.coastlines(resolution='50m', color='cyan', linewidth=0.3)

plt.title(f"{strs['platform_name']} | {strs['name']}\n{strs['start_time']:%Y-%m-%d %H:%M} UTC", fontweight='bold', fontsize=10, loc='left')
plt.title(f"{strs['area'].area_id}", fontweight='bold', fontsize=10, loc='right')


In [ ]:
var = get_enhanced_image(nscn['true_color']).data
var = var.transpose('y', 'x', 'bands')

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=200)
strs = var.attrs
abi_crs = var.attrs['area'].to_cartopy_crs()
ax = fig.add_subplot(1, 1, 1, projection=abi_crs)
ax.imshow(var.data, extent=(var.x[0], var.x[-1], var.y[-1], var.y[0]), origin='upper')

#ax.set_extent([-123.0, -74.5, 10.0, 37.0]) #cambiar a area geografica de interes

ax.coastlines(resolution='50m', color='cyan', linewidth=0.3)

plt.title(f"{strs['platform_name']} | {strs['name']}\n{strs['start_time']:%Y-%m-%d %H:%M} UTC", fontweight='bold', fontsize=10, loc='left')
plt.title(f"{strs['area'].area_id}", fontweight='bold', fontsize=10, loc='right')